In [ ]:
import pandas as pd
import numpy as np
import pytz
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from bumpplot import bumpchart
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy.interpolate import UnivariateSpline
pd.set_option('display.max_columns', None)